In [624]:
#Base de dados: Covid19br (https://github.com/wcota/covid19br/)

In [625]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as npy
from numpy.random import rand



def human_format(num):
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    # add more suffixes if you need them
    return '%.2f%s' % (num, ['', 'K', 'M', 'G', 'T', 'P'][magnitude])

covid_dataset_cities = pd.read_csv('c:\\Users\\gbdfigu\\Desktop\\Ada_Project_Avancado\\dataset\\cases-brazil-cities.csv')
covid_dataset_state = pd.read_csv('c:\\Users\\gbdfigu\\Desktop\\Ada_Project_Avancado\\dataset\\cases-brazil-states.csv')
covid_dataset_total = pd.read_csv('c:\\Users\\gbdfigu\\Desktop\\Ada_Project_Avancado\\dataset\\cases-brazil-total.csv')
covid_dataset_time_total = pd.read_csv('c:\\Users\\gbdfigu\\Desktop\\Ada_Project_Avancado\\dataset\\cases-brazil-cities-time_changesOnly.csv')

In [626]:
covid_dataset_clean = covid_dataset_time_total[covid_dataset_time_total['state'] != 'TOTAL']
#covid_dataset_clean = covid_dataset_time_total[covid_dataset_time_total['state'] == 'SP']
covid_dataset_clean = covid_dataset_clean[covid_dataset_clean['city'] != 'CASO SEM LOCALIZAÇÃO DEFINIDA']
pattern = 'CASO SEM LOCALIZAÇÃO DEFINIDA.*'
covid_dataset_clean = covid_dataset_clean[~covid_dataset_clean['city'].str.contains(pattern)]

covid_dataset_cities = covid_dataset_cities.drop(columns=['ibgeID', 'deaths_per_100k_inhabitants', 'totalCases_per_100k_inhabitants', 'deaths_by_totalCases', '_source', 'date', 'newCases', 'newDeaths', 'last_info_date'])
covid_dataset_clean

,epi_week,date,country,state,city,ibgeID,newDeaths,deaths,newCases,totalCases,deaths_per_100k_inhabitants,totalCases_per_100k_inhabitants,deaths_by_totalCases,_source,last_info_date
0,9,2020-02-25,Brazil,SP,São Paulo/SP,3550308,0,0,1,1,0.000,0.008,0.000,SES,2021-12-06
2,9,2020-02-28,Brazil,SP,São Paulo/SP,3550308,0,0,1,2,0.000,0.016,0.000,SES,2021-12-06
4,10,2020-03-04,Brazil,SP,São Paulo/SP,3550308,0,0,1,3,0.000,0.024,0.000,SES,2021-12-06
6,10,2020-03-05,Brazil,RJ,Barra Mansa/RJ,3300407,0,0,1,1,0.000,0.540,0.000,SES,2021-10-02
7,10,2020-03-05,Brazil,SP,São Paulo/SP,3550308,0,0,3,6,0.000,0.048,0.000,SES,2021-12-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2096355,311,2023-03-18,Brazil,SC,Águas Mornas/SC,4200606,0,13,-5,1214,195.606,18266.627,0.011,MS,2023-03-18
2096356,311,2023-03-18,Brazil,SP,Águas da Prata/SP,3500402,0,39,4,2174,472.041,26313.241,0.018,MS,2023-03-18
2096357,311,2023-03-18,Brazil,SP,Águas de Lindóia/SP,3500501,0,67,15,5614,354.347,29691.136,0.012,MS,2023-03-18
2096358,311,2023-03-18,Brazil,BA,Érico Cardoso/BA,2900504,0,53,13,2467,504.138,23466.185,0.021,MS,2023-03-18


In [627]:
max = covid_dataset_cities['totalCases'].max()
covid_dataset_cities[covid_dataset_cities['totalCases'] == max]

,country,state,city,deaths,totalCases
4225,Brazil,RJ,Rio de Janeiro/RJ,38144,1316242


In [628]:
covid_dataset_cities = covid_dataset_cities.groupby(['state','country']).sum()

In [629]:
top_10_states_cases =covid_dataset_cities.nlargest(n=10, columns=['totalCases'], keep='all').drop(columns=['city'])
top_10_states_cases

,,deaths,totalCases
state,country,,
SP,Brazil,179039,6469442
MG,Brazil,65507,4189760
RS,Brazil,41921,2962414
PR,Brazil,46022,2920177
RJ,Brazil,76852,2754359
SC,Brazil,22703,1986447
GO,Brazil,28041,1900459
BA,Brazil,31459,1792572
CE,Brazil,28157,1453417


In [630]:
covid_dataset_clean = covid_dataset_clean[['state', 'newCases', 'date']]
covid_dataset_clean['date'] = pd.to_datetime(covid_dataset_clean['date'], format='ISO8601')
covid_dataset_clean['ano'] = covid_dataset_clean['date'].dt.year.astype(str)
covid_dataset_clean = covid_dataset_clean[covid_dataset_clean['ano'] == '2022']
covid_dataset_clean['mes_ano'] = covid_dataset_clean['date'].dt.strftime('%b-%Y')
#covid_dataset_clean = covid_dataset_clean[['state','newCases','mes_ano']].groupby(['state', 'mes_ano'], as_index=False).sum()
covid_dataset_clean = covid_dataset_clean[['newCases','mes_ano']].groupby(['mes_ano'], as_index=False).sum()
covid_dataset_clean.sort_values(by='mes_ano', inplace = True)
covid_dataset_clean['mes_ano'] = covid_dataset_clean['mes_ano'].astype(str)
covid_dataset_clean

C:\Users\gbdfigu\AppData\Local\Temp\ipykernel_2452\912403699.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid_dataset_clean['date'] = pd.to_datetime(covid_dataset_clean['date'], format='ISO8601')
C:\Users\gbdfigu\AppData\Local\Temp\ipykernel_2452\912403699.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid_dataset_clean['ano'] = covid_dataset_clean['date'].dt.year.astype(str)


,state,newCases,date,ano,mes_ano
1659371,RS,6,2022-04-01,2022,Apr-2022
1681704,PR,2,2022-04-20,2022,Apr-2022
1681703,RS,1,2022-04-20,2022,Apr-2022
1681702,SP,-2,2022-04-20,2022,Apr-2022
1681701,MG,0,2022-04-20,2022,Apr-2022
...,...,...,...,...,...
1925145,PE,19,2022-09-19,2022,Sep-2022
1925146,PE,3,2022-09-19,2022,Sep-2022
1925147,BA,1,2022-09-19,2022,Sep-2022
1925138,GO,3,2022-09-19,2022,Sep-2022


In [631]:

covid_dataset_clean

,state,newCases,date,ano,mes_ano
1659371,RS,6,2022-04-01,2022,Apr-2022
1681704,PR,2,2022-04-20,2022,Apr-2022
1681703,RS,1,2022-04-20,2022,Apr-2022
1681702,SP,-2,2022-04-20,2022,Apr-2022
1681701,MG,0,2022-04-20,2022,Apr-2022
...,...,...,...,...,...
1925145,PE,19,2022-09-19,2022,Sep-2022
1925146,PE,3,2022-09-19,2022,Sep-2022
1925147,BA,1,2022-09-19,2022,Sep-2022
1925138,GO,3,2022-09-19,2022,Sep-2022


In [632]:
fig = plt.figure(figsize=(22,8))

sns.lineplot(data=covid_dataset_clean, x='mes_ano', y='newCases', marker='o', linewidth = 3, markersize=15, color='#29ADB2')
plt.fill_between(covid_dataset_clean['mes_ano'], covid_dataset_clean['newCases'], color='#C5E898', alpha=0.6)

for x, y in zip(covid_dataset_clean['mes_ano'], covid_dataset_clean['newCases']):
    plt.text(x = x, y = y-150, s = human_format(y), fontsize=12, color = 'black', ha='left', va='center',
             bbox=dict(facecolor='grey', boxstyle='round', alpha=0.3))

plt.show()


KeyboardInterrupt: 